In [65]:
#Sudoku raw_MY version: code version 1. 17 05 2023. (Problem in v1= Taking much longer to solve hard and harder sudoku).


import pandas as pd
import numpy as np
import time

# Function to convert string to 9x9 matrix
def str_to_matrix(s):
    # Convert string to list of digits
    digit_list = [int(d) for d in s]
    # Convert list to 9x9 numpy matrix
    return np.array(digit_list).reshape(9, 9)

# Ask user for CSV file path
csv_path = input("Please enter the path to the CSV file: ")

# Read CSV file
df = pd.read_csv(csv_path)

# Get the column names dynamically
puzzles_col = df.columns[0]
solutions_col = df.columns[1]

# Apply the function to the puzzles and solutions columns
df[puzzles_col] = df[puzzles_col].apply(str_to_matrix)
df[solutions_col] = df[solutions_col].apply(str_to_matrix)


# Check the results
#print(df.head())

def create_index_dicts():
    row_indices = {i: i // 9 for i in range(81)}
    column_indices = {i: i % 9 for i in range(81)}
    sub_box_indices = {i: (i // 27 * 3 + (i % 9) // 3) for i in range(81)}
    return row_indices, column_indices, sub_box_indices

def solve_sudoku(puzzle, row_indices, column_indices, sub_box_indices):
    def is_valid(puzzle, row, col, num):
        # Check if the number already exists in the row
        if num in puzzle[row]:
            return False

        # Check if the number already exists in the column
        for i in range(9):
            if num == puzzle[i][col]:
                return False

        # Check if the number already exists in the sub-box
        start_row = 3 * (row // 3)
        start_col = 3 * (col // 3)
        for i in range(start_row, start_row + 3):
            for j in range(start_col, start_col + 3):
                if num == puzzle[i][j]:
                    return False

        return True

    def solve(puzzle):
        for i in range(81):
            row = row_indices[i]
            col = column_indices[i]
            if puzzle[row][col] == 0:
                for num in range(1, 10):
                    if is_valid(puzzle, row, col, num):
                        puzzle[row][col] = num
                        if solve(puzzle):
                            return True
                        puzzle[row][col] = 0  # Backtrack
                return False
        return True

    # Create a copy of the puzzle to avoid modifying the original
    solved_puzzle = [row[:] for row in puzzle]

    # Solve the puzzle
    if solve(solved_puzzle):
        return solved_puzzle
    else:
        return None

# Initialize counters
total_puzzles = len(df)
total_iterations = 0
correctly_solved_puzzles = 0

# Initialize time tracking
start_time = time.time()

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    # Get the puzzle and solution dynamically
    puzzle = row[puzzles_col]
    solution = row[solutions_col]

    # Create the index dictionaries
    row_indices, column_indices, sub_box_indices = create_index_dicts()

    # Solve the Sudoku puzzle
    solved_puzzle = solve_sudoku(puzzle, row_indices, column_indices, sub_box_indices)

    # Update counters and check accuracy
    total_iterations += 1
    if solved_puzzle is not None and np.array_equal(solved_puzzle, solution):
        correctly_solved_puzzles += 1

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Calculate accuracy
accuracy = (correctly_solved_puzzles / total_puzzles) * 100

# Print results
print(f"Total Puzzles: {total_puzzles}")
print(f"Correctly Solved Puzzles: {correctly_solved_puzzles}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Total Iterations: {total_iterations}")
print(f"Time Used: {elapsed_time:.2f} seconds")
print (solved_puzzle, solution)   #test_sample (last row)

Please enter the path to the CSV file:  /home/ccmb/Downloads/easy.csv


Total Puzzles: 1000
Correctly Solved Puzzles: 1000
Accuracy: 100.00%
Total Iterations: 1000
Time Used: 3.08 seconds
[array([1, 2, 8, 6, 5, 9, 7, 4, 3]), array([5, 7, 6, 2, 4, 3, 1, 9, 8]), array([3, 9, 4, 8, 1, 7, 5, 6, 2]), array([9, 4, 7, 1, 3, 6, 2, 8, 5]), array([2, 8, 5, 9, 7, 4, 6, 3, 1]), array([6, 1, 3, 5, 2, 8, 4, 7, 9]), array([4, 3, 2, 7, 9, 1, 8, 5, 6]), array([7, 6, 1, 3, 8, 5, 9, 2, 4]), array([8, 5, 9, 4, 6, 2, 3, 1, 7])] [[1 2 8 6 5 9 7 4 3]
 [5 7 6 2 4 3 1 9 8]
 [3 9 4 8 1 7 5 6 2]
 [9 4 7 1 3 6 2 8 5]
 [2 8 5 9 7 4 6 3 1]
 [6 1 3 5 2 8 4 7 9]
 [4 3 2 7 9 1 8 5 6]
 [7 6 1 3 8 5 9 2 4]
 [8 5 9 4 6 2 3 1 7]]


In [37]:
#Sudoku FASTER version:  code version 2. 18 05 2023. Modification in v1=Changed the solving method, but continued with my dictionaries concept in it. (Problemn in v2- contains unneccesary steps to convert from string-list-matrix, and back).

import pandas as pd
import numpy as np
import time

# Function to convert string to 9x9 matrix
def str_to_matrix(s):
    # Convert string to list of digits
    digit_list = [int(d) for d in s]
    # Convert list to 9x9 numpy matrix
    return np.array(digit_list).reshape(9, 9)

def solve_sudoku(puzzle):
    digits = '123456789'
    rows = 'ABCDEFGHI'
    cols = digits

    def cross(a, b):
        return [s + t for s in a for t in b]

    positions = cross(rows, cols)
    unittuple = (
        [cross(rows, c) for c in cols] +
        [cross(r, cols) for r in rows] +
        [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
    )
    units = dict((s, [u for u in unittuple if s in u]) for s in positions)
    peers = dict((s, set(sum(units[s], [])) - {s}) for s in positions)

    def parse_string(string):
        values = dict((s, digits) for s in positions)
        for s, d in string_values(string).items():
            if d in digits and not assign(values, s, d):
                return False
        return values

    def string_values(string):
        chars = np.array(list(string)).astype(str)
        assert len(chars) == 81
        return dict(zip(positions, chars))

    def assign(values, s, d):
        other_values = values[s].replace(d, '')
        if all(eliminate(values, s, d2) for d2 in other_values):
            return values
        else:
            return False

    def eliminate(values, s, d):
        if d not in values[s]:
            return values
        values[s] = values[s].replace(d, '')
        if len(values[s]) == 0:
            return False
        elif len(values[s]) == 1:
            d2 = values[s]
            if not all(eliminate(values, s2, d2) for s2 in peers[s]):
                return False
        for u in units[s]:
            dplaces = [s for s in u if d in values[s]]
            if len(dplaces) == 0:
                return False
            elif len(dplaces) == 1:
                if not assign(values, dplaces[0], d):
                    return False
        return values

    def display(values):
        width = 1 + max(len(values[s]) for s in positions)
        line = '+'.join(['-' * (width * 3)] * 3)
        for r in rows:
            print(''.join(values[r + c].center(width) + ('|' if c in '36' else '') for c in cols))
            if r in 'CF':
                print(line)
        print()

    def search(values):
        if values is False:
            return False
        if all(len(values[s]) == 1 for s in positions):
            return values
        n, s = min((len(values[s]), s) for s in positions if len(values[s]) > 1)
        return some(search(assign(values.copy(), s, d)) for d in values[s])

    def some(seq):
        for e in seq:
            if e:
                return e
        return False

    # Solve the puzzle
    values = parse_string(puzzle)
    if values:
        return search(values)
    else:
        return None

# Ask user for CSV file path
csv_path = input("Please enter the path to the CSV file: ")

# Read CSV file
df = pd.read_csv(csv_path)

# Get the column names dynamically
puzzles_col = df.columns[0]
solutions_col = df.columns[1]

# Apply the function to the puzzles and solutions columns
df[puzzles_col] = df[puzzles_col].apply(str_to_matrix)
df[solutions_col] = df[solutions_col].apply(str_to_matrix)

# Initialize counters
total_puzzles = len(df)
total_iterations = 0
correctly_solved_puzzles = 0

# Initialize time tracking
start_time = time.time()

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    # Get the puzzle and solution dynamically
    puzzle = row[puzzles_col]
    solution = row[solutions_col]

    # Solve the Sudoku puzzle
    puzzle_str = ''.join(str(d) for d in puzzle.flat)
    solved_puzzle = solve_sudoku(puzzle_str)

    # Convert the solved puzzle back to a matrix
    if solved_puzzle is not None:
        solved_puzzle = np.array(list(solved_puzzle.values())).reshape(9, 9).astype(int)

    # Update counters and check accuracy
    total_iterations += 1
    if solved_puzzle is not None and np.array_equal(solved_puzzle, solution):
        correctly_solved_puzzles += 1

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Calculate accuracy
accuracy = (correctly_solved_puzzles / total_puzzles) * 100

# Print results
print(f"Total Puzzles: {total_puzzles}")
print(f"Correctly Solved Puzzles: {correctly_solved_puzzles}")
print(f"Accuracy: {accuracy:.3f}%")
print(f"Total Iterations: {total_iterations}")
print(f"Time Used: {elapsed_time:.8f} seconds")
print (solved_puzzle, solution)    #test_sample (last row)


Please enter the path to the CSV file:  /home/ccmb/Downloads/easy.csv


Total Puzzles: 1000
Correctly Solved Puzzles: 1000
Accuracy: 100.000%
Total Iterations: 1000
Time Used: 2.57263374 seconds
[[1 2 8 6 5 9 7 4 3]
 [5 7 6 2 4 3 1 9 8]
 [3 9 4 8 1 7 5 6 2]
 [9 4 7 1 3 6 2 8 5]
 [2 8 5 9 7 4 6 3 1]
 [6 1 3 5 2 8 4 7 9]
 [4 3 2 7 9 1 8 5 6]
 [7 6 1 3 8 5 9 2 4]
 [8 5 9 4 6 2 3 1 7]] [[1 2 8 6 5 9 7 4 3]
 [5 7 6 2 4 3 1 9 8]
 [3 9 4 8 1 7 5 6 2]
 [9 4 7 1 3 6 2 8 5]
 [2 8 5 9 7 4 6 3 1]
 [6 1 3 5 2 8 4 7 9]
 [4 3 2 7 9 1 8 5 6]
 [7 6 1 3 8 5 9 2 4]
 [8 5 9 4 6 2 3 1 7]]


In [58]:
#Sudoku JET version: code version 3. 19 05 2023. Modification in v2= Time of string-list-matrix (to & fro) conversion saved (not significant for 1k puzzles. May make difference while solving millions of puzzles). Solving method same.

import pandas as pd
import numpy as np
import time

def solve_sudoku(puzzle):
    digits = '123456789'
    rows = 'ABCDEFGHI'
    cols = digits

    def cross(a, b):
        return [s + t for s in a for t in b]

    positions = cross(rows, cols)
    unittuple = (
        [cross(rows, c) for c in cols] +
        [cross(r, cols) for r in rows] +
        [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
    )
    units = dict((s, [u for u in unittuple if s in u]) for s in positions)
    peers = dict((s, set(sum(units[s], [])) - {s}) for s in positions)

    def parse_string(string):
        values = dict((s, digits) for s in positions)
        for s, d in string_values(string).items():
            if d in digits and not assign(values, s, d):
                return False
        return values

    def string_values(string):
        chars = np.array(list(string)).astype(str)
        assert len(chars) == 81
        return dict(zip(positions, chars))

    def assign(values, s, d):
        other_values = values[s].replace(d, '')
        if all(eliminate(values, s, d2) for d2 in other_values):
            return values
        else:
            return False

    def eliminate(values, s, d):
        if d not in values[s]:
            return values
        values[s] = values[s].replace(d, '')
        if len(values[s]) == 0:
            return False
        elif len(values[s]) == 1:
            d2 = values[s]
            if not all(eliminate(values, s2, d2) for s2 in peers[s]):
                return False
        for u in units[s]:
            dplaces = [s for s in u if d in values[s]]
            if len(dplaces) == 0:
                return False
            elif len(dplaces) == 1:
                if not assign(values, dplaces[0], d):
                    return False
        return values

    def display(values):
        width = 1 + max(len(values[s]) for s in positions)
        line = '+'.join(['-' * (width * 3)] * 3)
        for r in rows:
            print(''.join(values[r + c].center(width) + ('|' if c in '36' else '') for c in cols))
            if r in 'CF':
                print(line)
        print()

    def search(values):
        if values is False:
            return False
        if all(len(values[s]) == 1 for s in positions):
            return values
        n, s = min((len(values[s]), s) for s in positions if len(values[s]) > 1)
        return some(search(assign(values.copy(), s, d)) for d in values[s])

    def some(seq):
        for e in seq:
            if e:
                return e
        return False

    # Solve the puzzle
    values = parse_string(puzzle)
    if values:
        return search(values)
    else:
        return None

# Ask user for CSV file path
csv_path = input("Please enter the path to the CSV file: ")

# Read CSV file
df = pd.read_csv(csv_path)

# Get the column names dynamically
puzzles_col = df.columns[0]
solutions_col = df.columns[1]

# Initialize counters
total_puzzles = len(df)
total_iterations = 0
correctly_solved_puzzles = 0

# Initialize time tracking
start_time = time.time()

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    # Get the puzzle and solution dynamically
    puzzle = row[puzzles_col]
    solution = row[solutions_col]

    # Solve the Sudoku puzzle
    solved_puzzle = solve_sudoku(puzzle)
    
    # Convert the solved puzzle (returned-values) from dict-to-string.
    if solved_puzzle is not None:
        solved_puzzle = ''.join(map(str, list(solved_puzzle.values())))
    
    # Update counters and check accuracy
    total_iterations += 1
    if solved_puzzle is not None and np.array_equal(solved_puzzle, solution):
        correctly_solved_puzzles += 1

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Calculate accuracy
accuracy = (correctly_solved_puzzles / total_puzzles) * 100

# Print results
print(f"Total Puzzles: {total_puzzles}")
print(f"Correctly Solved Puzzles: {correctly_solved_puzzles}")
print(f"Accuracy: {accuracy:.3f}%")
print(f"Total Iterations: {total_iterations}")
print(f"Time Used: {elapsed_time:.8f} seconds")
print (solved_puzzle, solution)   #test_sample (last row)


Please enter the path to the CSV file:  /home/ccmb/Downloads/easy.csv


Total Puzzles: 1000
Correctly Solved Puzzles: 1000
Accuracy: 100.000%
Total Iterations: 1000
Time Used: 2.48657513 seconds
128659743576243198394817562947136285285974631613528479432791856761385924859462317 128659743576243198394817562947136285285974631613528479432791856761385924859462317


In [75]:
#Sudoku SPACE_JET version :code version 4. 22 05 2023. Modification in v3= optimization in v3 methods & calls & Commands.  

import pandas as pd
import numpy as np
import time

def solve_sudoku(puzzle):
    digits = '123456789'
    rows = 'ABCDEFGHI'
    cols = digits

    def cross(a, b):
        return [s + t for s in a for t in b]

    positions = cross(rows, cols)
    unittuple = (
        [cross(rows, c) for c in cols] +
        [cross(r, cols) for r in rows] +
        [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
    )
    units = dict((s, [u for u in unittuple if s in u]) for s in positions)
    peers = dict((s, set(sum(units[s], [])) - {s}) for s in positions)

    def parse_string(string):
        values = dict((s, digits) for s in positions)
        for s, d in string_values(string).items():
            if d in digits and not assign(values, s, d):
                return False
        return values

    def string_values(string):
        chars = np.array(list(string)).astype(str)
        assert len(chars) == 81
        return dict(zip(positions, chars))

    def assign(values, s, d):
        other_values = values[s].replace(d, '')
        if all(eliminate(values, s, d2) for d2 in other_values):
            return values
        else:
            return False

    def eliminate(values, s, d):
        if d not in values[s]:
            return values
        values[s] = values[s].replace(d, '')
        if len(values[s]) == 0:
            return False
        elif len(values[s]) == 1:
            d2 = values[s]
            if not all(eliminate(values, s2, d2) for s2 in peers[s]):
                return False
        for u in units[s]:
            dplaces = [s for s in u if d in values[s]]
            if len(dplaces) == 0:
                return False
            elif len(dplaces) == 1:
                if not assign(values, dplaces[0], d):
                    return False
        return values

    def search(values):
        if values is False:
            return False
        if all(len(values[s]) == 1 for s in positions):
            return ''.join(values[s] for s in positions)  # Combine values into a single string
        n, s = min((len(values[s]), s) for s in positions if len(values[s]) > 1)
        return some(search(assign(values.copy(), s, d)) for d in values[s])

    def some(seq):
        for e in seq:
            if e:
                return e
        return False

    # Solve the puzzle
    values = parse_string(puzzle)
    if values:
        return search(values)
    else:
        return None
    
# Ask user for CSV file path
csv_path = input("Please enter the path to the CSV file: ")

# Read CSV file
df = pd.read_csv(csv_path)

# Get the column names dynamically
puzzles_col = df.columns[0]
solutions_col = df.columns[1]

# Initialize counters
total_puzzles = len(df)
total_iterations = 0
correctly_solved_puzzles = 0

# Initialize time tracking
start_time = time.time()

# Iterate over the DataFrame rows
for index, row in df.iterrows():

    # Solve the Sudoku puzzle
    solved_puzzle = solve_sudoku(row[puzzles_col])

    # Update counters and check accuracy
    total_iterations += 1
    if solved_puzzle is not None and np.array_equal(solved_puzzle, row[solutions_col]):
        correctly_solved_puzzles += 1

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Calculate accuracy
accuracy = (correctly_solved_puzzles / total_puzzles) * 100

# Print results
print(f"Total Puzzles: {total_puzzles}")
print(f"Correctly Solved Puzzles: {correctly_solved_puzzles}")
print(f"Accuracy: {accuracy:.3f}%")
print(f"Total Iterations: {total_iterations}")
print(f"Time Used: {elapsed_time:.8f} seconds")
print (solved_puzzle, row[solutions_col])   #test_sample (last row)


# /home/ccmb/Downloads/easy.csv

Please enter the path to the CSV file:  /home/ccmb/Downloads/easy.csv


Total Puzzles: 1000
Correctly Solved Puzzles: 1000
Accuracy: 100.000%
Total Iterations: 1000
Time Used: 2.50704789 seconds
128659743576243198394817562947136285285974631613528479432791856761385924859462317 128659743576243198394817562947136285285974631613528479432791856761385924859462317


In [78]:
#Sudoku teJET version :code version 5. 22 05 2023. Modification in v4= Added command to use multiple core of cpu to run code (my Ideas, ChatGPT's coding). NO change in v4 solution logic/method.

import multiprocessing

import pandas as pd
import numpy as np
import time
from multiprocessing import Pool

def solve_sudoku(puzzle):
    digits = '123456789'
    rows = 'ABCDEFGHI'
    cols = digits

    def cross(a, b):
        return [s + t for s in a for t in b]

    positions = cross(rows, cols)
    unittuple = (
        [cross(rows, c) for c in cols] +
        [cross(r, cols) for r in rows] +
        [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
    )
    units = dict((s, [u for u in unittuple if s in u]) for s in positions)
    peers = dict((s, set(sum(units[s], [])) - {s}) for s in positions)

    def parse_string(string):
        values = dict((s, digits) for s in positions)
        for s, d in string_values(string).items():
            if d in digits and not assign(values, s, d):
                return False
        return values
                                                                #
    def string_values(string):
        chars = np.array(list(string)).astype(str)
        assert len(chars) == 81
        return dict(zip(positions, chars))
                                                                    #
    def assign(values, s, d):
        other_values = values[s].replace(d, '')
        if all(eliminate(values, s, d2) for d2 in other_values):
            return values
        else:
            return False
                                                                    #
    def eliminate(values, s, d):
        if d not in values[s]:
            return values
        values[s] = values[s].replace(d, '')
        if len(values[s]) == 0:
            return False
        elif len(values[s]) == 1:
            d2 = values[s]
            if not all(eliminate(values, s2, d2) for s2 in peers[s]):
                return False
        for u in units[s]:
            dplaces = [s for s in u if d in values[s]]
            if len(dplaces) == 0:
                return False
            elif len(dplaces) == 1:
                if not assign(values, dplaces[0], d):
                    return False
        return values

    def search(values):
        if values is False:
            return False
        if all(len(values[s]) == 1 for s in positions):
            return ''.join(values[s] for s in positions)  # Combine values into a single string
        n, s = min((len(values[s]), s) for s in positions if len(values[s]) > 1)
        return some(search(assign(values.copy(), s, d)) for d in values[s])

    def some(seq):
        for e in seq:
            if e:
                return e
        return False

    # Solve the puzzle
    values = parse_string(puzzle)
    if values:
        return search(values)
    else:
        return None
        

# Define a worker function for parallel processing
def solve_sudoku_worker(puzzle):
    return solve_sudoku(puzzle)

# Ask user for CSV file path
csv_path = input("Please enter the path to the CSV file: ")

# Read CSV file
df = pd.read_csv(csv_path)

# Get the column names dynamically
puzzles_col = df.columns[0]
solutions_col = df.columns[1]

# Initialize counters
total_puzzles = len(df)
total_iterations = 0
correctly_solved_puzzles = 0

# Initialize time tracking
start_time = time.time()

# Create a multiprocessing Pool with the number of desired CPU cores
num_cores = multiprocessing.cpu_count()
pool = Pool(processes=num_cores)

# Iterate over the DataFrame rows and solve puzzles in parallel
results = pool.map(solve_sudoku_worker, df[puzzles_col])

# Close the pool to free resources
pool.close()
pool.join()

# Iterate over the results and update counters
for index, row in df.iterrows():
    solved_puzzle = results[index]

    total_iterations += 1
    if solved_puzzle is not None and np.array_equal(solved_puzzle, row[solutions_col]):
        correctly_solved_puzzles += 1

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Calculate accuracy
accuracy = (correctly_solved_puzzles / total_puzzles) * 100

# Print results
print(f"Total Puzzles: {total_puzzles}")
print(f"Correctly Solved Puzzles: {correctly_solved_puzzles}")
print(f"Accuracy: {accuracy:.3f}%")
print(f"Total Iterations: {total_iterations}")
print(f"Time Used: {elapsed_time:.8f} seconds")
print(solved_puzzle, row[solutions_col])  # Test sample (last row)

#/home/ccmb/Downloads/harder.csv

Please enter the path to the CSV file:  /home/ccmb/Downloads/easy.csv


Total Puzzles: 1000
Correctly Solved Puzzles: 1000
Accuracy: 100.000%
Total Iterations: 1000
Time Used: 0.43300223 seconds
128659743576243198394817562947136285285974631613528479432791856761385924859462317 128659743576243198394817562947136285285974631613528479432791856761385924859462317


In [ ]:
#Sudoku teJET pro version :code version 6. Modification in v5= changes in v4 (=v5) core logic of sulving method, making it more faster (removing input puzzle accuracy checks, converting string into dictionary bypassing numpy.array, solving multiple puzzles at once in zip format).... whilw keeping miluticore mode as same in v5. 


